### 한국어 Corpus 전처리
- 데이터셋: Korpora에서 로딩
- 형태소 분석기 설정
- 단어 사전 설정

In [2]:
# load module
from Korpora import Korpora
from konlpy.tag import *
import spacy

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import *                   # 자연어처리 단어사전 관련 모듈
from torch.utils.data import DataLoader, Dataset


c:\Users\PC\anaconda3\envs\nlp_38_018\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\PC\anaconda3\envs\nlp_38_018\lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [7]:
import pandas as pd
import numpy as np

In [4]:
# load data
nsmc = Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\PC\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\PC\Kor

In [5]:
nsmc.train[0]

LabeledSentence(text='아 더빙.. 진짜 짜증나네요 목소리', label=0)

In [6]:
nsmc.test[0]

LabeledSentence(text='굳 ㅋ', label=1)

In [8]:
nsmc_df = pd.DataFrame(nsmc.test)

nsmc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [9]:
nsmc_df.head()

,text,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [10]:
nsmc_df.tail()

,text,label
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0
49999,마무리는 또 왜이래,0


- custom dataset
---

In [23]:
# 단어 사전을 생성 시 활용
class TextDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
        self.rows = feature.shape[0]
    
    def __len__(self):
        return self.rows
        
    
    def __getitem__(self, index):
        return self.feature.iloc[index], self.label.iloc[index]

In [24]:
# nams dataset instance
nsmc_ds = TextDataset(nsmc_df['text'], nsmc_df['label'])

In [25]:
for f, l in nsmc_ds:
    print(f, l)
    break

굳 ㅋ 1


- 텍스트 전처리
---
- 토큰화
- 정제 (불용어, 구두점, 오타)
- 단어사전 생성
- 문장 수치화

In [26]:
# create tokenizer
okt_tokenizer = get_tokenizer(Okt().morphs)
spacy_tokenizer = get_tokenizer('spacy', language='ko_core_news_sm')

In [27]:
# 토큰화를 진행해주는 generator
# dataset에서 처리할 dataset만 로딩해서 토큰화 진행하는 반복자 함수
def gen_token(dataset):
    for text, label in dataset:
        # 불용어 처리 코드를 여기에 넣어도 되고
        # 위에서 생성하기 전에 함수를 하나 선언해도 됨
        
        yield okt_tokenizer(text)

In [28]:
# 단어 사전 생성
UNK_TOKEN = '<UNK>'
PAD_TOKEN = '<PAD>'
nsmc_vocab = build_vocab_from_iterator(iterator=gen_token(nsmc_ds), specials=[UNK_TOKEN])

nsmc_vocab.set_default_index(nsmc_vocab[UNK_TOKEN])

In [36]:
nsmc_vocab.lookup_tokens([10, 100])

['도', 'ㅋ']